<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Steam games dataset 2024
</div>

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
<strong>Extraccion de informacion</strong>

<p>
Recuperdor de información, para saber  que tipo de contenido se desea como resumenes, detallesm reseñas, edades
</p>
</div>

In [2]:
import spacy_transformers
import pandas as pd
import re
import spacy
import textacy
import visualise_spacy_tree
import glob
from spacy.matcher import Matcher 
import os
import sys
import nltk
import en_core_web_trf
import re
from pathlib import Path
from pprint import pprint
from sklearn_crfsuite import CRF
from nltk.corpus import opinion_lexicon
from spacy.matcher import Matcher
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from sklearn_crfsuite import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

import numpy as np
from collections import defaultdict
from collections import Counter

/home/natashalop/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/natashalop/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
pd.set_option('display.max_colwidth', 2000)
pd.options.display.max_rows=100

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Realizamos esta linea ya que teniamos que definir cada columna con su tipo de dato para no tener problemas luego
</div>

In [4]:
dtype_dict = {
    'AppID': 'int64',
    'name': 'object',
    'release_date': 'object',
    'required_age': 'object',
    'price': 'float64',
    'dlc_count': 'float64',
    'detailed_description': 'object',
    'about_the_game': 'object',
    'short_description': 'object',
    'reviews': 'object',
    'header_image': 'object',
    'website': 'object',
    'support_url': 'object',
    'support_email': 'object',
    'windows': 'object',
    'mac': 'object',
    'linux': 'object',
    'metacritic_score': 'object',
    'metacritic_url': 'object',
    'achievements': 'object',
    'recommendations': 'object',
    'notes': 'object',
    'supported_languages': 'object',
    'full_audio_languages': 'object',
    'packages': 'object',
    
    'Unnamed: 87': 'float64',
    'Unnamed: 88': 'float64',
    'Unnamed: 89': 'float64'
}


<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Leemos el dataset 
</div>

In [5]:
df = pd.read_csv('games_may2024_cleaned.csv',  dtype=dtype_dict, low_memory=False)

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Elegimos las columnas que queremos extraer su informacion
</div>

In [6]:
df = df[[ "about_the_game", "short_description","genres", "supported_languages"]]

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Realizamos estas 2 celdas ya que no queremos los NAN en cada fila de cada columna
</div>

In [7]:
df_cleaned = df.dropna()

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Podemos ver el df sin los NAN en cada fila de cada columna 
</div>

In [8]:
df_cleaned

,about_the_game,short_description,genres,supported_languages
0,"For over two decades, Counter-Strike has offered an elite competitive experience, one shaped by millions of players from across the globe. And now the next chapter in the CS story is about to begin. This is Counter-Strike 2. A free upgrade to CS:GO, Counter-Strike 2 marks the largest technical leap in Counter-Strike’s history. Built on the Source 2 engine, Counter-Strike 2 is modernized with realistic physically-based rendering, state of the art networking, and upgraded Community Workshop tools. In addition to the classic objective-focused gameplay that Counter-Strike pioneered in 1999, Counter-Strike 2 features: All-new CS Ratings with the updated Premier mode Global and Regional leaderboards Upgraded and overhauled maps Game-changing dynamic smoke grenades Tick-rate-independent gameplay Redesigned visual effects and audio All items from CS:GO moving forward to CS2","For over two decades, Counter-Strike has offered an elite competitive experience, one shaped by millions of players from across the globe. And now the next chapter in the CS story is about to begin. This is Counter-Strike 2.","['Action', 'Free to Play']","['Czech', 'Danish', 'Dutch', 'English', 'Finnish', 'French', 'German', 'Hungarian', 'Italian', 'Japanese', 'Korean', 'Norwegian', 'Polish', 'Portuguese - Portugal', 'Portuguese - Brazil', 'Romanian', 'Russian', 'Simplified Chinese', 'Spanish - Spain', 'Swedish', 'Thai', 'Traditional Chinese', 'Turkish', 'Bulgarian', 'Ukrainian', 'Greek', 'Spanish - Latin America', 'Vietnamese']"
1,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS for free. Land on strategic locations, loot weapons and supplies, and survive to become the last team standing across various, diverse Battlegrounds. Squad up and join the Battlegrounds for the original Battle Royale experience that only PUBG: BATTLEGROUNDS can offer. This content download will also provide access to the BATTLEGROUNDS Test Server, which requires a separate download to play. Optional in-game purchases available.","Play PUBG: BATTLEGROUNDS for free. Land on strategic locations, loot weapons and supplies, and survive to become the last team standing across various, diverse Battlegrounds. Squad up and join the Battlegrounds for the original Battle Royale experience that only PUBG: BATTLEGROUNDS can offer.","['Action', 'Adventure', 'Massively Multiplayer', 'Free to Play']","['English', 'Korean', 'Simplified Chinese', 'French', 'German', 'Spanish - Spain', 'Arabic', 'Japanese', 'Polish', 'Portuguese - Portugal', 'Russian', 'Turkish', 'Thai', 'Italian', 'Portuguese - Brazil', 'Traditional Chinese', 'Ukrainian']"
2,"The most-played game on Steam. Every day, millions of players worldwide enter battle as one of over a hundred Dota heroes. And no matter if it's their 10th hour of play or 1,000th, there's always something new to discover. With regular updates that ensure a constant evolution of gameplay, features, and heroes, Dota 2 has truly taken on a life of its own. One Battlefield. Infinite Possibilities. When it comes to diversity of heroes, abilities, and powerful items, Dota boasts an endless array—no two games are the same. Any hero can fill multiple roles, and there's an abundance of items to help meet the needs of each game. Dota doesn't provide limitations on how to play, it empowers you to express your own style. All heroes are free. Competitive balance is Dota's crown jewel, and to ensure everyone is playing on an even field, the core content of the game—like the vast pool of heroes—is available to all players. Fans can collect cosmetics for heroes and fun add-ons for the world they inhabit, but everything you need to play is already included before you join your first match. Bring your friends and party up. Dota is deep, and constantly evolving, but it's never too late to join. Learn the ropes playing co-op vs. bots. Sharpen your skills in the hero demo mode. Jump into the behavior- and skill-based matchmak

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Realizamos la limpieza profunda
</div>

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Instalamos la libreria de emoji para poder eliminar cada emoji que aparezca en el dataset
</div>

In [15]:
#%pip install emoji

In [9]:
import re

def clean_df(df):
    if df is None:
        return None
    df_copy = df.copy()
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002600-\U000026FF"  # Miscellaneous Symbols
                               u"\U00002700-\U000027BF"  # Dingbats
                               "]+", flags=re.UNICODE)
    
    # Define the replacement patterns
    replace_patterns = [
        ('•', False),
        ('◆', False),
        ('™', False),
        ('®', False),
        ('©', False),
        ('█', False),
        ('▒', False),
        ('__', False),
        ('……', False),
        ('†', False),
        ('※', False),
        ('', False),
        ('£', False),
        ('⊰⊱', False),
        ('๑̀́و', False),
        ("'", False),
        ('...', False),
        ('-', False),
        ('—', False),
        ('_', False),
        ('___________', False),
        (r'[,.:;=()*+&^%$#@!{}~\[\]<>?/|\\\-\'"«»–“”]', True)
    ]
    
    for columna in df_copy.columns:
        # Apply emoji pattern removal
        df_copy[columna] = df_copy[columna].astype(str).apply(lambda x: emoji_pattern.sub(r'', x))
        
        # Apply all replacement patterns
        for pattern, is_regex in replace_patterns:
            df_copy[columna] = df_copy[columna].str.replace(pattern, '', regex=is_regex)
        
        # Convert text to lowercase
        df_copy[columna] = df_copy[columna].str.lower()
    
    return df_copy


In [10]:
df_cleaned = clean_df(df_cleaned)

In [11]:
# Esto nos ayuda a saber si el signo o caracter esta todavia en la columna del dataset
contiene_caracter = df_cleaned['about_the_game'].astype(str).str.contains('•')

# Verifica si algún elemento contiene el carácter "®"
if contiene_caracter.any():
    print("La columna contiene al menos un '®'.")
else:
    print("La columna no contiene el carácter '®'.")

La columna no contiene el carácter '®'.


In [12]:
df_cleaned

,about_the_game,short_description,genres,supported_languages
0,for over two decades counterstrike has offered an elite competitive experience one shaped by millions of players from across the globe and now the next chapter in the cs story is about to begin this is counterstrike 2 a free upgrade to csgo counterstrike 2 marks the largest technical leap in counterstrike’s history built on the source 2 engine counterstrike 2 is modernized with realistic physicallybased rendering state of the art networking and upgraded community workshop tools in addition to the classic objectivefocused gameplay that counterstrike pioneered in 1999 counterstrike 2 features allnew cs ratings with the updated premier mode global and regional leaderboards upgraded and overhauled maps gamechanging dynamic smoke grenades tickrateindependent gameplay redesigned visual effects and audio all items from csgo moving forward to cs2,for over two decades counterstrike has offered an elite competitive experience one shaped by millions of players from across the globe and now the next chapter in the cs story is about to begin this is counterstrike 2,action free to play,czech danish dutch english finnish french german hungarian italian japanese korean norwegian polish portuguese portugal portuguese brazil romanian russian simplified chinese spanish spain swedish thai traditional chinese turkish bulgarian ukrainian greek spanish latin america vietnamese
1,land loot survive play pubg battlegrounds for free land on strategic locations loot weapons and supplies and survive to become the last team standing across various diverse battlegrounds squad up and join the battlegrounds for the original battle royale experience that only pubg battlegrounds can offer this content download will also provide access to the battlegrounds test server which requires a separate download to play optional ingame purchases available,play pubg battlegrounds for free land on strategic locations loot weapons and supplies and survive to become the last team standing across various diverse battlegrounds squad up and join the battlegrounds for the original battle royale experience that only pubg battlegrounds can offer,action adventure massively multiplayer free to play,english korean simplified chinese french german spanish spain arabic japanese polish portuguese portugal russian turkish thai italian portuguese brazil traditional chinese ukrainian
2,the mostplayed game on steam every day millions of players worldwide enter battle as one of over a hundred dota heroes and no matter if its their 10th hour of play or 1000th theres always something new to discover with regular updates that ensure a constant evolution of gameplay features and heroes dota 2 has truly taken on a life of its own one battlefield infinite possibilities when it comes to diversity of heroes abilities and powerful items dota boasts an endless arrayno two games are the same any hero can fill multiple roles and theres an abundance of items to help meet the needs of each game dota doesnt provide limitations on how to play it empowers you to express your own style all heroes are free competitive balance is dotas crown jewel and to ensure everyone is playing on an even field the core content of the gamelike the vast pool of heroesis available to all players fans can collect cosmetics for heroes and fun addons for the world they inhabit but everything you need to play is already included before you join your first match bring your friends and party up dota is deep and constantly evolving but its never too late to join learn the ropes playing coop vs bots sharpen your skills in the hero demo mode jump into the behavior and skillbased matchmaking system that ensures youll be matched with the right players each game,every day millions of players worldwide enter battle as one of over a hundred dota heroes and no matter if its their 10th hour of play or 1000th theres always something new to discover with regular updates that ensure a

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Entrenamiento
</div>

In [15]:
nlp = spacy.load('en_core_web_sm')

/home/natashalop/.local/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [16]:
def extraer_terminos_clave(texto):
    doc = nlp(texto)
    palabras_filtradas = [token.text for token in doc if not token.is_stop and token.is_alpha]
    return palabras_filtradas

In [17]:
palabras = []

for chunk in pd.read_csv('games_may2024_cleaned.csv', chunksize=1000):
    for texto in chunk['about_the_game']:
        if pd.notna(texto):
            palabra = extraer_terminos_clave((str)(texto))
            palabras.extend(palabra)


In [18]:
palabras_unidas = ' '.join(palabras)

In [19]:
nlp.max_length = 1500000  # Aumenta el límite de caracteres permitido

# Función para dividir el texto en fragmentos manejables
def dividir_texto(texto, n=100000):
    """Divide el texto en fragmentos de n caracteres."""
    for i in range(0, len(texto), n):
        yield texto[i:i + n]

In [20]:
# Divide el texto en fragmentos más pequeños
fragmentos = list(dividir_texto(palabras_unidas, n=100000))

# Inicializa un Counter para los términos clave
contador_terminos = Counter()

# Procesa cada fragmento y extrae términos clave con textacy
for fragmento in fragmentos:
    doc = textacy.make_spacy_doc(fragmento, lang='en_core_web_lg')
    keyterms = textacy.extract.keyterms.textrank(doc, topn=100)
    contador_terminos.update([term for term, _ in keyterms])

# Extrae los términos clave más comunes
terminos_clave = contador_terminos.most_common(10)

# Imprime los términos clave
print("Términos clave más comunes:", terminos_clave)

In [ ]:
# sentences = textacy.make_spacy_doc(palabras_unidas, lang='en_core_web_lg')
# keyterms=textacy.extract.keyterms.textrank(sentences,topn=100)

In [15]:
df_cleaned.columns

Index(['about_the_game', 'short_description', 'genres', 'supported_languages'], dtype='object')

In [17]:
# Función para convertir el dataframe a un formato compatible con spaCy
def df_to_spacy_format(df):
    spacy_format = []
    for _, row in df.iterrows():
        spacy_format.append(row['about_the_game'])
    return spacy_format

# Convertir el dataframe a formato spaCy
df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# Guardar el dataframe en formato spaCy en un archivo de texto
df_cleaned_file_path = 'df_cleaned_about_the_game.txt'
with open(df_cleaned_file_path, 'w') as file:
    for entry in df_cleaned_spacy_format:
        file.write(entry + '\n')

In [18]:
# Función para convertir el dataframe a un formato compatible con spaCy
def df_to_spacy_format(df):
    spacy_format = []
    for _, row in df.iterrows():
        spacy_format.append(row['short_description'])
    return spacy_format

# Convertir el dataframe a formato spaCy
df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# Guardar el dataframe en formato spaCy en un archivo de texto
df_cleaned_file_path = 'df_cleaned_short_description.txt'
with open(df_cleaned_file_path, 'w') as file:
    for entry in df_cleaned_spacy_format:
        file.write(entry + '\n')

In [19]:
# Función para convertir el dataframe a un formato compatible con spaCy
def df_to_spacy_format(df):
    spacy_format = []
    for _, row in df.iterrows():
        spacy_format.append(row['genres'])
    return spacy_format

# Convertir el dataframe a formato spaCy
df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# Guardar el dataframe en formato spaCy en un archivo de texto
df_cleaned_file_path = 'df_cleaned_genres.txt'
with open(df_cleaned_file_path, 'w') as file:
    for entry in df_cleaned_spacy_format:
        file.write(entry + '\n')

In [20]:
# Función para convertir el dataframe a un formato compatible con spaCy
def df_to_spacy_format(df):
    spacy_format = []
    for _, row in df.iterrows():
        spacy_format.append(row['supported_languages'])
    return spacy_format

# Convertir el dataframe a formato spaCy
df_cleaned_spacy_format = df_to_spacy_format(df_cleaned)

# Guardar el dataframe en formato spaCy en un archivo de texto
df_cleaned_file_path = 'df_supported_languages.txt'
with open(df_cleaned_file_path, 'w') as file:
    for entry in df_cleaned_spacy_format:
        file.write(entry + '\n')